In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import matplotlib as mat
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
%matplotlib inline

pd.options.display.max_colwidth = 100

import random
import os

from numpy.random import seed
seed(42)

random.seed(42)
os.environ['PYTHONHASHSEED'] = str(42)
os.environ['TF_DETERMINISTIC_OPS'] = '1'

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import glob
import cv2

from tensorflow.random import set_seed
set_seed(42)

import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd

# Replace 'your_file.csv' with the actual file name and path
file_path = '/content/drive/MyDrive/chest_xray'

train_path = os.path.join(file_path,"train")
test_path=os.path.join(file_path,"test")

train_normal = glob.glob(train_path+"/NORMAL/*.jpeg")
train_pneumonia = glob.glob(train_path+"/PNEUMONIA/*.jpeg")

test_normal = glob.glob(test_path+"/NORMAL/*.jpeg")
test_pneumonia = glob.glob(test_path+"/PNEUMONIA/*.jpeg")


In [4]:
train_list = [x for x in train_normal]
train_list.extend([x for x in train_pneumonia])

df_train = pd.DataFrame(np.concatenate([['Normal']*len(train_normal) , ['Pneumonia']*len(train_pneumonia)]), columns = ['class'])
df_train['image'] = [x for x in train_list]

test_list = [x for x in test_normal]
test_list.extend([x for x in test_pneumonia])

df_test = pd.DataFrame(np.concatenate([['Normal']*len(test_normal) , ['Pneumonia']*len(test_pneumonia)]), columns = ['class'])
df_test['image'] = [x for x in test_list]

In [7]:
df_train

,class,image
0,Normal,/content/drive/MyDrive/chest_xray/train/NORMAL/NORMAL-3342765-0001.jpeg
1,Normal,/content/drive/MyDrive/chest_xray/train/NORMAL/NORMAL-3269314-0001.jpeg
2,Normal,/content/drive/MyDrive/chest_xray/train/NORMAL/NORMAL-3342765-0002.jpeg
3,Normal,/content/drive/MyDrive/chest_xray/train/NORMAL/NORMAL-3250852-0001.jpeg
4,Normal,/content/drive/MyDrive/chest_xray/train/NORMAL/NORMAL-3526928-0001.jpeg
...,...,...
5227,Pneumonia,/content/drive/MyDrive/chest_xray/train/PNEUMONIA/BACTERIA-4057077-0001.jpeg
5228,Pneumonia,/content/drive/MyDrive/chest_xray/train/PNEUMONIA/BACTERIA-3955228-0001.jpeg
5229,Pneumonia,/content/drive/MyDrive/chest_xray/train/PNEUMONIA/BACTERIA-4371619-0002.jpeg
5230,Pneumonia,/content/drive/MyDrive/chest_xray/train/PNEUMONIA/BACTERIA-387616-0001.jpeg


In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score, precision_score, recall_score

img_width, img_height = 150, 150
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=df_train,
    x_col="image",
    y_col="class",
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode="binary",
    shuffle=True,
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=df_test,
    x_col="image",
    y_col="class",
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode="binary",
    shuffle=False,
)

model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())


model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))


model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


epochs = 4

history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=test_generator,
)

# Make predictions on the test set
predictions = model.predict(test_generator)
predicted_classes = (predictions > 0.5).astype("int32")

# Get true labels from the test generator
true_labels = test_generator.classes

# Calculate metrics
accuracy = accuracy_score(true_labels, predicted_classes)
precision = precision_score(true_labels, predicted_classes)
recall = recall_score(true_labels, predicted_classes)

# Print the results
print(f'Test Accuracy: {accuracy:.4f}')
print(f'Test Precision: {precision:.4f}')
print(f'Test Recall: {recall:.4f}')


Found 5232 validated image filenames belonging to 2 classes.
Found 661 validated image filenames belonging to 2 classes.
Epoch 1/4
164/164 [==============================] - 108s 617ms/step - loss: 0.4058 - accuracy: 0.8190 - val_loss: 0.3094 - val_accuracy: 0.8759
Epoch 2/4
164/164 [==============================] - 101s 614ms/step - loss: 0.2546 - accuracy: 0.8926 - val_loss: 0.3276 - val_accuracy: 0.8684
Epoch 3/4
164/164 [==============================] - 98s 595ms/step - loss: 0.2255 - accuracy: 0.9102 - val_loss: 0.2941 - val_accuracy: 0.8487
Epoch 4/4
21/21 [==============================] - 6s 273ms/step
Test Accuracy: 0.9032
Test Precision: 0.8789
Test Recall: 0.9859
